In [1]:
% matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import model_list, models, fitting
import corner

from utils import rj2cmb

In [9]:
mean_beta = 1.6
mean_temp = 20.
sigma_beta = .2
sigma_temp = 4.

DUST_I = 50.
DUST_P = 5. / 1.41
amp_I=rj2cmb(353e9, DUST_I)
amp_Q=rj2cmb(353e9, DUST_P)
amp_U=rj2cmb(353e9, DUST_P)

nu = np.logspace(np.log10(30), np.log10(500), 7) * 1e9

In [4]:
test = model_list.prob1mbb_model
control = model_list.dust_model
cmb = model_list.cmb_model

In [6]:
models_fit = [control, cmb] 
amp_names = []
param_names = []

for mod in models_fit:
        # Parameter names
        amp_names += ["%s_%s" % (mod.model, pol) for pol in "IQU"]
        param_names += mod.param_names

In [7]:
pnames_MBB = amp_names + param_names
pnames_MBB

['mbb_I', 'mbb_Q', 'mbb_U', 'cmb_I', 'cmb_Q', 'cmb_U', 'dust_beta', 'dust_T']

In [10]:
runs = 100

fsigma_T=1.
fsigma_P=10e3
beam_mat = np.identity(3*len(nu)) # Beam model
params_MBB = [control.amp_I, control.amp_Q, control.amp_U, cmb.amp_I, cmb.amp_Q, cmb.amp_U, control.dust_beta,
              control.dust_T]

initial_vals_MBB = (amp_I, amp_Q, amp_U, control.amp_U, cmb.amp_I, cmb.amp_Q, mean_beta, mean_temp)

parent_model = 'mbb'

logps = np.empty(runs)
covs = np.empty(runs, dtype=object)
best_fits = np.empty(runs, dtype=object)

for i in range(runs):
    
    D_vec_MBB, Ninv = fitting.generate_data(nu, fsigma_T, fsigma_P, [test, cmb])
    #D_vec_probMBB, Ninv = fitting.generate_data(nu, fsigma_T, fsigma_P, [test, cmb])

    data_spec_MBB = (nu, D_vec_MBB, Ninv, beam_mat)
    #data_spec_probMBB = (nu, D_vec_probMBB, Ninv, beam_mat)

    p_spec_MBB = (pnames_MBB, initial_vals_MBB, parent_model)
    #p_spec_probMBB = (pnames_probMBB, initial_vals_probMBB, parent_model)
    
    pnames_out_control, samples_control, logp_control  = fitting.joint_mcmc(data_spec_MBB, [control, cmb], p_spec_MBB, nwalkers=20, 
               burn=1000, steps=10000, nthreads=2, sample_file=None)
    
    logps[i] = logp_control.max()
    covs[i] = np.cov(samples_control)
    best_fits[i] = [samples_control[j][logp_control.argmax()] for j in range(len(pnames_MBB))]
    

    # pnames_out_mixed, samples_mixed, logp_mixed  = fitting.joint_mcmc(data_spec_probMBB, [control, cmb], p_spec_MBB, nwalkers=20, 
    #          burn=1000, steps=10000, nthreads=2, sample_file=None)


Unhandled exception in thread started by <bound method Thread.__bootstrap of <Thread(Thread-2, stopped daemon 123145431834624)>>


IndexError: string index out of range